In [22]:
import os
import re
import time
import imageio
import json
import glob
from pathlib import Path
from datetime import datetime, timedelta

import geojsoncontour
import numpy as np
import pandas as pd

from netCDF4 import Dataset as NetCDFFile
import matplotlib.pyplot as plt

from wrf import getvar, interplevel, to_np, latlon_coords

import plotly.graph_objects as go

In [2]:
nc_file = NetCDFFile('../wrf_output')
time_size = nc_file.dimensions['Time'].size

In [3]:
height = getvar(nc_file, 'height', timeidx=1)
Var_all = getvar(nc_file, 'tc', timeidx=1)

var = interplevel(Var_all, height, 200)

print(var)

(lats, lons) = latlon_coords(var)

<xarray.DataArray 'temp_interp' (south_north: 81, west_east: 90)>
array([[26.655838, 26.658287, 26.657635, ..., 26.972134, 26.994387,
        27.016954],
       [26.6094  , 26.674477, 26.720581, ..., 26.928345, 26.952646,
        26.99092 ],
       [26.56189 , 26.655165, 26.764233, ..., 26.898792, 26.9301  ,
        26.964827],
       ...,
       [24.470493, 24.469011, 24.48296 , ..., 26.246532, 26.25344 ,
        26.282623],
       [24.467682, 24.470179, 24.481556, ..., 26.248093, 26.236677,
        26.257603],
       [24.46515 , 24.467947, 24.470707, ..., 26.25211 , 26.237888,
        26.232687]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 -74.96 -74.96 ... -74.68 -74.68
    XLAT     (south_north, west_east) float32 10.85 10.85 10.85 ... 11.11 11.11
    XTIME    float32 180.0
    Time     datetime64[ns] 2022-03-29T21:00:00
    level    int64 200
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:      104
    units:        

In [41]:
def get_data(nc_file: NetCDFFile, timeidx:int):
    dx = 150
    height = getvar(nc_file, 'height', timeidx=timeidx)

    u_all = getvar(nc_file, 'ua', timeidx=timeidx)
    v_all = getvar(nc_file, 'va', timeidx=timeidx)
    w_all = getvar(nc_file, 'wa', timeidx=timeidx)
    T_all = getvar(nc_file, 'tc', timeidx=timeidx)
    P_all = getvar(nc_file, 'pressure', timeidx=timeidx)

    P = interplevel(P_all, height, dx)
    T = interplevel(T_all, height, dx)
    u = interplevel(u_all, height, dx)
    v = interplevel(v_all, height, dx)
    w = interplevel(w_all, height, dx)
    
    data = {
        'wind': ('Wind velocity (m/s)', np.sqrt(u ** 2 + v ** 2 + w ** 2)),
        'temp': ('Temperature (C)', T),
        'uwind': ('U wind velocity (m/s)', u),
        'vwind': ('V wind velocity (m/s)', v),
        'press': ('Pressure (hPa)', P)
    }

    return data

def geojson_title_to_float(title):
    result = re.search(r"([-]?([0-9]*[.])?[0-9]+)-([-]?([0-9]*[.])?[0-9]+)", title)
    groups = result.groups()
    
    value = np.median([float(groups[0]), float(groups[2])])
    
    return value

def gj_to_df(gj):
    gj_data = np.zeros([len(gj['features']), 2])

    for i in range(len(gj['features'])):
        gj['features'][i]['id'] = i
        gj_data[i, 0] = i
        gj_data[i, 1] = geojson_title_to_float(gj['features'][i]['properties']['title'])

    df = pd.DataFrame(gj_data, columns=['id', 'variable'])
    
    return df


def get_folium(nc_file: NetCDFFile, timeidx: int, nc_var: str, start_date: datetime):
    date = start_date + timedelta(hours=timeidx * 3) - timedelta(hours=5)

    data = get_data(nc_file, timeidx)

    (caption, variable) = data[nc_var]
    (lats, lons) = latlon_coords(variable)

    contour = plt.contourf(lons, lats, variable, cmap=plt.cm.jet)

    gj = json.loads(geojsoncontour.contourf_to_geojson(contourf=contour, ndigits=4, unit='m'))
    df_contour = gj_to_df(gj)

    trace = go.Choroplethmapbox(
        geojson=gj,
        locations=df_contour.id,
        z=df_contour.variable,
        colorscale='jet',
        marker_line_width=0.1,
        marker=dict(opacity=0.2)
    )
    
    layout = go.Layout(
        title=f"{caption} - {date}",
        title_x=0.5,
        width=600,
        margin=dict(t=26, b=0, l=0, r=0),
        font=dict(color='dark grey', size=10),
        mapbox = dict(
            center = dict(
                lat = lats.mean().item(0),
                lon = lons.mean().item(0)
            ),
            zoom=11,
            style='carto-positron'
        )
    )
    
    fig = go.Figure(data=[trace], layout=layout)
    
    return fig

def get_image(timeidx: int, nc_var: str, start_date: datetime):
    png_file = f"{nc_var}_{timeidx}.png"
    
    fig = get_folium(nc_file, timeidx, nc_var, start_date)
    fig.write_image(png_file)

    img = imageio.imread(png_file)

    os.remove(png_file)

    return img

In [42]:
nc_var = 'temp'
start_date = datetime.strptime("2022-03-26 18", '%Y-%m-%d %H')

results = [get_image(timeidx, nc_var, start_date) for timeidx in range(time_size)]

imageio.mimwrite(f"{nc_var}_test.gif", results, fps=1);